# 从零训练一个小的bloom模型

注意事项: transformers版本有问题，
目前使用这个是可以的
`pip install transformers==4.45.2`
版本有问题

```requirements.txt
absl-py==2.1.0
accelerate==1.2.1
aiohappyeyeballs==2.4.4
aiohttp==3.11.11
aiosignal==1.3.2
anyio==4.7.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==3.0.0
async-lru==2.0.4
attrs==24.3.0
babel==2.16.0
beautifulsoup4==4.12.3
bleach==6.2.0
certifi==2024.12.14
cffi==1.17.1
charset-normalizer==3.4.0
comm==0.2.2
datasets==3.2.0
debugpy==1.8.11
decorator==5.1.1
defusedxml==0.7.1
dill==0.3.8
executing==2.1.0
fastjsonschema==2.21.1
filelock==3.16.1
fqdn==1.5.1
frozenlist==1.5.0
fsspec==2024.9.0
grpcio==1.68.1
h11==0.14.0
httpcore==1.0.7
httpx==0.28.1
huggingface-hub==0.27.0
idna==3.10
ipykernel==6.29.5
ipython==8.31.0
ipywidgets==8.1.5
isoduration==20.11.0
jedi==0.19.2
Jinja2==3.1.4
json5==0.10.0
jsonpointer==3.0.0
jsonschema==4.23.0
jsonschema-specifications==2024.10.1
jupyter==1.1.1
jupyter-console==6.6.3
jupyter-events==0.11.0
jupyter-lsp==2.2.5
jupyter_client==8.6.3
jupyter_core==5.7.2
jupyter_server==2.14.2
jupyter_server_terminals==0.5.3
jupyterlab==4.3.4
jupyterlab_pygments==0.3.0
jupyterlab_server==2.27.3
jupyterlab_widgets==3.0.13
Markdown==3.7
MarkupSafe==3.0.2
matplotlib-inline==0.1.7
mistune==3.0.2
mpmath==1.3.0
multidict==6.1.0
multiprocess==0.70.16
nbclient==0.10.2
nbconvert==7.16.4
nbformat==5.10.4
nest-asyncio==1.6.0
networkx==3.4.2
notebook==7.3.1
notebook_shim==0.2.4
numpy==2.2.0
nvidia-cublas-cu12==12.4.5.8
nvidia-cuda-cupti-cu12==12.4.127
nvidia-cuda-nvrtc-cu12==12.4.127
nvidia-cuda-runtime-cu12==12.4.127
nvidia-cudnn-cu12==9.1.0.70
nvidia-cufft-cu12==11.2.1.3
nvidia-curand-cu12==10.3.5.147
nvidia-cusolver-cu12==11.6.1.9
nvidia-cusparse-cu12==12.3.1.170
nvidia-nccl-cu12==2.21.5
nvidia-nvjitlink-cu12==12.4.127
nvidia-nvtx-cu12==12.4.127
overrides==7.7.0
packaging==24.2
pandas==2.2.3
pandocfilters==1.5.1
parso==0.8.4
pexpect==4.9.0
platformdirs==4.3.6
prometheus_client==0.21.1
prompt_toolkit==3.0.48
propcache==0.2.1
protobuf==5.29.2
psutil==6.1.1
ptyprocess==0.7.0
pure_eval==0.2.3
pyarrow==18.1.0
pycparser==2.22
Pygments==2.18.0
python-dateutil==2.9.0.post0
python-json-logger==3.2.1
pytz==2024.2
PyYAML==6.0.2
pyzmq==26.2.0
referencing==0.35.1
regex==2024.11.6
requests==2.32.3
rfc3339-validator==0.1.4
rfc3986-validator==0.1.1
rpds-py==0.22.3
safetensors==0.4.5
Send2Trash==1.8.3
setuptools==75.6.0
six==1.17.0
sniffio==1.3.1
soupsieve==2.6
stack-data==0.6.3
sympy==1.13.1
tensorboard==2.18.0
tensorboard-data-server==0.7.2
terminado==0.18.1
tinycss2==1.4.0
tokenizers==0.20.3
torch==2.5.1
tornado==6.4.2
tqdm==4.67.1
traitlets==5.14.3
transformers==4.45.2
triton==3.1.0
types-python-dateutil==2.9.0.20241206
typing_extensions==4.12.2
tzdata==2024.2
uri-template==1.3.0
urllib3==2.2.3
wcwidth==0.2.13
webcolors==24.11.1
webencodings==0.5.1
websocket-client==1.8.0
Werkzeug==3.1.3
widgetsnbextension==4.0.13
xxhash==3.5.0
yarl==1.18.3
```

In [1]:
from transformers import BloomConfig, BloomForCausalLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForLanguageModeling, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

In [ ]:
ds = load_dataset("shaowenchen/wiki_zh")
print(ds)

In [ ]:
ds = ds["train"].train_test_split(test_size=10, seed=42)
ds

In [ ]:
# 由于bloom自带的分词器有25w 这里为了简单，用一下Bert-base-chinese的分词器  事实上会有很多问题(OOV等)，但是这样可以节省资源
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-chinese", model_max_length=2048)  # 长度修改一下 毕竟生成模型
tokenizer

In [ ]:
special_tokens_dict = {"bos_token": "[bos_token]", "eos_token": "[eos_token]"}
tokenizer.add_special_tokens(special_tokens_dict)
tokenizer.vocab_size

In [ ]:
bloom_config = BloomConfig(vocab_size=tokenizer.vocab_size+len(special_tokens_dict), hidden_size=1536, n_layer=18, n_head=12, bos_token_id=tokenizer.bos_token_id, eos_token_id=tokenizer.eos_token_id)
bloom_config

In [ ]:
model = BloomForCausalLM(bloom_config)
model

In [ ]:
sum(param.numel() for name, param in model.named_parameters())

简单看下生成效果

In [ ]:
input_text = tokenizer.bos_token + "孙中山"
print(input_text)
inputs = tokenizer(input_text, return_tensors="pt", add_special_tokens=False)["input_ids"]
print(inputs)
# inputs = {k:v.to(model.device) for k, v in inputs.items() if k != "token_type_ids"}[]
output = model.generate(inputs, max_new_tokens=20)
print(output)
print(tokenizer.decode(output[0].cpu().numpy().tolist()))

# 正式开始

In [ ]:
sample_data = ds["train"].select(range(10))
sample_data

In [11]:
def processFuntion(example):
    example = [tokenizer.bos_token + text + tokenizer.eos_token for text in example["text"]]
    result = tokenizer(example, truncation=True, add_special_tokens=False)
    return result

In [ ]:
tokenizer_sample_data = sample_data.map(processFuntion, batched=True, remove_columns=sample_data.column_names)
tokenizer_sample_data

In [ ]:
input_ids = tokenizer_sample_data[0]["input_ids"]
tokenizer.decode(input_ids)

In [ ]:
ds = ds.map(processFuntion, batched=True, remove_columns=ds["train"].column_names)
ds

In [15]:
args = TrainingArguments(
    output_dir="output/generate_model_train",
    max_steps=12000,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    fp16=True,
    learning_rate=1e-4,
    logging_steps=10,
    save_strategy="steps",
    save_total_limit=3,
    warmup_steps=2000,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    tokenizer=tokenizer
)

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset=ds["train"], batch_size=2, collate_fn=DataCollatorForLanguageModeling(tokenizer, mlm=False))
for item in dataloader:
    print(item.keys())
    break

In [ ]:
trainer.train()

# 看下训练完的效果

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
checkpoint = "output/generate_model_train/checkpoint-4000"
model = AutoModelForCausalLM.from_pretrained(checkpoint).to("cuda:0")
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
input_text = tokenizer.bos_token + "孙中山是一位"
print(input_text)
inputs = tokenizer(input_text, return_tensors="pt", add_special_tokens=False)["input_ids"].to(model.device)
print(inputs)
# inputs = {k:v.to(model.device) for k, v in inputs.items() if k != "token_type_ids"}[]
output = model.generate(inputs, max_new_tokens=100)
print(output)
print(tokenizer.decode(output[0].cpu().numpy().tolist()))